## Linear System

Consider the following linear system.
\begin{align}
    \frac{d x_1}{dt}&= x_1-4x_2,\\
    \frac{d x_2}{dt}&= 4 x_1-7 x_2.
\end{align}
(Example 2 on the paper)

In [23]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [10]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Concatenate, Add, Multiply,Average, Subtract
from tensorflow.keras.models import Model
import traffic_data_generator
import traffic_util

Fix the random seeds in numpy, tensorlfow

In [11]:
import random as rn

rand_seed=116
rn.seed(rand_seed)
np.random.seed(rand_seed)
tf.random.set_seed(rand_seed)


In [27]:
example_name="traffic"
data_dir=traffic_data_generator.set_up(example_name)

In [28]:

delta_gen=0.05
n_traj=5000
n_samples=100000


In [29]:
#===========================
#     Output directories 
#===========================
experiment_name="_no_share"+ \
     "_"+str(n_samples)+"_Delta_"+str(delta_gen)


model_dir=example_name+"/"+experiment_name
plot_dir=model_dir+"/Plots"
checkpoint_dir=model_dir+"/check_points"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

In [22]:
ith_run=0
domain=np.load(data_dir+'/parameters AW'+str(ith_run)+ 'd_gen='+str(delta_gen) + 'n_samples='+str(n_samples)+'.npy')
traj_len=int(np.random.uniform(domain[9,0],domain[9,1])) # total time 
n_traj_per_run=10
n_runs=int(np.ceil(n_traj/n_traj_per_run))
traj_dir=data_dir+ '/parameters AW'+str(ith_run)+ 'd_gen='+str(delta_gen) + 'n_samples='+str(n_samples)
assert os.path.exists(traj_dir)
#parameter check
domain

array([[3.00e+02, 3.00e+02],
       [5.00e+00, 5.00e+00],
       [1.00e+00, 1.00e+00],
       [8.00e-01, 8.00e-01],
       [1.30e+00, 1.30e+00],
       [1.15e+00, 1.15e+00],
       [1.00e-01, 1.00e-01],
       [4.00e-01, 4.00e-01],
       [1.00e-01, 1.00e-01],
       [1.00e+02, 1.00e+02]])

In [44]:
#This returns an array with first column the location of the start of the bin, the second column as the density (number of cars)
#in that bin and the third as the average velocity of the cars in that bin. 
#NOTE the last entry for column 2 and 3 is always zero to allow us to define intervals well.
def indicator_func(x,v,bin_size,n_bins):
    bin_matrix=np.zeros([3,n_bins+1])
    bin_matrix[0]=-np.arange(n_bins+1)*bin_size
    ind=np.argsort(x)[::-1]
    bin_indicator=np.ceil(x[ind]/bin_size)
    v_sorted=v[ind]
    bin_end=np.zeros(n_bins+1,dtype=int)
    for i_bin in range(1,n_bins+1):
        bin_end[i_bin]=bin_end[i_bin-1]
        if -i_bin+1 in bin_indicator:
            bin_end[i_bin]=np.where(bin_indicator==-i_bin+1)[0][-1]
            bin_matrix[1,i_bin-1]=(bin_end[i_bin]-bin_end[i_bin-1]+(i_bin==1))/len(x)
            bin_matrix[2,i_bin-1]=np.mean(v_sorted[bin_end[i_bin-1]+(i_bin>1):bin_end[i_bin]+1])
    return bin_matrix

In [111]:
len_hist_MZ=20
Delta_t=0.5
n_bins=10
bin_size=13



# histogram_matrix=np.zeros([n_bin_time_step,3,n_bins+1])
# for i_bin_time in range(n_bin_time_step):
#     bin_time_int=int(round(Delta_t/delta_gen))*i_bin_time
#     histogram_matrix[i_bin_time,:,:]=indicator_func(loc_matrix[:,bin_time_int],vel_matrix[:,bin_time_int],bin_size,n_bins)

In [112]:
sample_per_traj=int(round(n_samples/n_traj))
delta_ratio=int(round(Delta_t/delta_gen))
y_index=np.ceil(np.random.uniform(len_hist_MZ+delta_ratio-1,traj_len,sample_per_traj)/delta_gen).astype(int)

In [116]:
sample_per_traj=int(round(n_samples/n_traj))
delta_ratio=int(round(Delta_t/delta_gen))
for i_run in range(n_runs): #range(n_runs):
    while not os.path.exists(traj_dir+ '/location'+str(i_run*n_traj_per_run) + ".npy"):
        time.sleep(200)
    location=np.load(traj_dir+ '/location'+str(i_run*n_traj_per_run) + ".npy")
    v=np.zeros_like(location[0,:,0])
    for i_traj in range(n_traj_per_run):
        y_index_vec=np.floor(np.random.uniform((len_hist_MZ+1)*Delta_t,traj_len,sample_per_traj)/delta_gen).astype(int)
        for i_sample in range(sample_per_traj):
            y_index=y_index_vec[i_sample]
            y_build=indicator_func(location[i_traj,:,y_index],v,bin_size,n_bins)[1,:-1]
            y_build=y_build.reshape([1,-1])
            x_build=indicator_func(location[i_traj,:,y_index-delta_ratio],v,bin_size,n_bins)[1,:-1]
            x_build=x_build.reshape([1,1,-1])
            for i_x in range(1,len_hist_MZ):
                x_temp=indicator_func(location[i_traj,:,y_index-delta_ratio*i_x],v,bin_size,n_bins)[1,:-1]
                x_temp=x_temp.reshape([1,1,-1])
                x_build=np.concatenate((x_build,x_temp),axis=1)
            if i_traj==0 and i_run==0 and i_sample==0:
                x=x_build
                y=y_build
            else:
                x=np.concatenate((x,x_build),axis=0)
                y=np.concatenate((y,y_build),axis=0)
    print(x.shape)
np.save(data_dir+'/x_delta_t='+str(Delta_t)+ 'MZ_hist='+str(len_hist_MZ) +'.npy', x)
np.save(data_dir+'/y_delta_t='+str(Delta_t)+ 'MZ_hist='+str(len_hist_MZ) +'.npy', y)


(200, 20, 10)
(400, 20, 10)
(600, 20, 10)
(800, 20, 10)
(1000, 20, 10)
(1200, 20, 10)
(1400, 20, 10)
(1600, 20, 10)
(1800, 20, 10)
(2000, 20, 10)
(2200, 20, 10)
(2400, 20, 10)
(2600, 20, 10)
(2800, 20, 10)
(3000, 20, 10)
(3200, 20, 10)
(3400, 20, 10)
(3600, 20, 10)
(3800, 20, 10)
(4000, 20, 10)
(4200, 20, 10)
(4400, 20, 10)
(4600, 20, 10)
(4800, 20, 10)
(5000, 20, 10)
(5200, 20, 10)
(5400, 20, 10)
(5600, 20, 10)
(5800, 20, 10)
(6000, 20, 10)
(6200, 20, 10)
(6400, 20, 10)
(6600, 20, 10)
(6800, 20, 10)
(7000, 20, 10)
(7200, 20, 10)
(7400, 20, 10)
(7600, 20, 10)
(7800, 20, 10)
(8000, 20, 10)
(8200, 20, 10)
(8400, 20, 10)
(8600, 20, 10)
(8800, 20, 10)
(9000, 20, 10)
(9200, 20, 10)
(9400, 20, 10)
(9600, 20, 10)
(9800, 20, 10)
(10000, 20, 10)
(10200, 20, 10)
(10400, 20, 10)
(10600, 20, 10)
(10800, 20, 10)
(11000, 20, 10)
(11200, 20, 10)
(11400, 20, 10)
(11600, 20, 10)
(11800, 20, 10)
(12000, 20, 10)
(12200, 20, 10)
(12400, 20, 10)
(12600, 20, 10)
(12800, 20, 10)
(13000, 20, 10)
(13200, 20, 1

In [134]:
#Building Reference Trajectories
location=np.load(data_dir + '/location i=' + str(ith_run) + 'd_gen='+str(delta_gen) + ".npy")
v=np.zeros_like(location[0,:,0])
for i_traj in range(location.shape[0]):
    ref_build=indicator_func(location[i_traj,:,0],v,bin_size,n_bins)[1,:-1]
    ref_build=ref_build.reshape([1,1,-1])
    for i_ref in range(0,int(round(traj_len/delta_gen))):
        ref_temp=indicator_func(location[i_traj,:,i_ref],v,bin_size,n_bins)[1,:-1]
        ref_temp=ref_temp.reshape([1,1,-1])
        ref_build=np.concatenate((ref_build,ref_temp),axis=1)
    if i_traj==0:
        ref=ref_build
    else:
        ref=np.concatenate((ref,ref_build),axis=0)
np.save(data_dir+'/ref_delta_t='+str(delta_gen)+'bin_n,l=' + str(n_bins)+'_'+str(bin_size) + '.npy', ref)


In [122]:
traj_len

100

In [135]:
ref.shape

(10, 2001, 10)

In [136]:
ref_build.shape

(1, 2001, 10)

In [133]:
ref_temp.shape

(1, 1, 10)